`Course Instructor`: **John Chiasson**

`Author (TA)`: **Ruthvik Vaila**

# Imports
* NOTE: Runs the cells with `tf.compat.v1.enable_eager_execution()` in order to examine the contents in the `.tfrecord` files that we previously generated.
* Tested on `Python 3.7.5` with `Tensorflow 1.15.0` and `Keras 2.2.4`. 
* Tested on `Python 2.7.17` with `Tensorflow 1.15.3` and `Keras 2.2.4`.

In [1]:
import sys, os
sys.version

'3.7.5 (default, Nov  7 2019, 10:50:52) \n[GCC 8.3.0]'

In [2]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.python.client import device_lib
import numpy as np
import sys, pickle, os
import h5py, time, inspect
import keras, IPython
import IPython.display as display
import keras, warnings
warnings.filterwarnings(action='once')
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
# this make sure thaat if using a gpu total gpu memory is not gobbled
# up by tensorflow and allows growth as required

Using TensorFlow backend.


In [ ]:
device_lib.list_local_devices()

In [3]:
tf.__version__

/home/visionteam/python37_env/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'1.15.0'

In [4]:
keras.__version__

'2.2.4'

# Read the raw dataset .tfrecord file
* `raw_data` is Tensor with some metadata
* `example` is a loaded string that has the protocol buffer data

In [5]:
filenames = ['10_tfrecords/train/EMNIST_train_strings_0.tfrecord']
raw_dataset = tf.data.TFRecordDataset(filenames, compression_type='GZIP',num_parallel_reads=1)
raw_dataset

<TFRecordDatasetV1 shapes: (), types: tf.string>

In [6]:
for raw_data in raw_dataset.take(1):
    print(raw_data)

tf.Tensor(b'\n\xdfq\n\x0e\n\x05label\x12\x05\x1a\x03\n\x01-\n\xccq\n\timage_raw\x12\xbeq\x12\xbbq\n\xb8q\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x000A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x00@A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@A\x00\x00@A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [7]:
for raw_data in raw_dataset.take(1):
    print(repr(raw_data))

<tf.Tensor: id=33, shape=(), dtype=string, numpy=b'\n\xdfq\n\x0e\n\x05label\x12\x05\x1a\x03\n\x01-\n\xccq\n\timage_raw\x12\xbeq\x12\xbbq\n\xb8q\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x000A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x00@A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@A\x00\x00@A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000A\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

* Get the protobuffer kind of data structure by passing the above string to `tf.train.Example.FromString`

In [8]:
example = tf.train.Example.FromString(raw_data.numpy())
print(example)

features {
  feature {
    key: "image_raw"
    value {
      float_list {
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 11.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 11.0
        value: 11.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 11.0
        value: 12.0
        value: 0.0
        value: 0.0
        value: 12.0
        value: 12.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 11.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 11.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
        value: 0.0
    

In [9]:
print(example.features.feature['image_raw'].float_list.value[0])
print(example.features.feature['label'].int64_list.value[0])

0.0
45


## Parsing using `tf.FixedLenFeature`

In [10]:
image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([3630], tf.float32),
}

* parsed image

In [11]:
tensor = tf.io.parse_single_example(raw_data, image_feature_description)['image_raw'].numpy()
print(tensor)
print
print(tensor.sum())
print
print(tf.io.parse_single_example(raw_data, image_feature_description)['label'].numpy())

[0. 0. 0. ... 0. 0. 0.]
2637.0
45


## Parsing using `tf.VarLenFeature()`
* Notice that `tf.io.VarLenFeature` returned a sparse tensor that needs to be converted to a dense one

In [12]:
image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.VarLenFeature(tf.float32),
}

* parsed image

In [14]:
sparse_tensor = tf.io.parse_single_example(raw_data, image_feature_description)['image_raw']
print(sparse_tensor)
dense_tensor = tf.sparse.to_dense(sparse_tensor)
print
print(dense_tensor.numpy().sum())
print
print(tf.io.parse_single_example(raw_data, image_feature_description)['label'].numpy())

SparseTensor(indices=tf.Tensor(
[[   0]
 [   1]
 [   2]
 ...
 [3627]
 [3628]
 [3629]], shape=(3630, 1), dtype=int64), values=tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(3630,), dtype=float32), dense_shape=tf.Tensor([3630], shape=(1,), dtype=int64))
2637.0
45


# Restart the notebook to free up the `GPU` and `RAM`.

In [ ]:
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel